In [30]:
import numpy as np
import pandas as pd

In [31]:
import torch
torch.__version__

'1.7.0'

In [32]:
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader,TensorDataset,random_split
from torchvision.transforms import transforms

In [33]:
def get_train_labels(train_label):
    for i,num in enumerate(train_label):
        if(num> 4.5):
            train_label[i] = 1
        else:
            train_label[i]= 0
            
    return train_label

In [34]:
with open('../input/sfft-data/SFFTFinalData.npy', 'rb') as training_data:
     X= np.load(training_data)

In [35]:
X.shape

(147966, 70)

In [36]:
with open('../input/sfft-data/SFFTFinalLabels.npy', 'rb') as training_labels:
     Y= np.load(training_labels)

In [37]:
Y.shape

(147966, 2)

In [38]:
arousal_labels = np.ravel(Y[:, [0]])
valence_labels = np.ravel(Y[:, [1]])

In [39]:
arousal_labels=get_train_labels(arousal_labels)

In [40]:
valence_labels = get_train_labels(valence_labels)

In [41]:
from sklearn.model_selection import train_test_split

In [42]:
class SFTT(nn.Module):

    def __init__(self):
        super().__init__()
        self.network = nn.Sequential(
            nn.Conv2d(1,64,kernel_size=3,stride=1,padding=1),
            nn.BatchNorm2d(64),
            nn.ReLU(),
            nn.MaxPool2d(2,2),
            nn.Conv2d(64,128,kernel_size=3,stride=1,padding=1),
            nn.BatchNorm2d(128),
            nn.ReLU(),
            nn.MaxPool2d(2,2),

            nn.Conv2d(128,256,kernel_size=3,stride=1,padding=1),
            nn.BatchNorm2d(256),
            nn.ReLU(),
            nn.AdaptiveAvgPool2d(1),

            nn.Flatten(),
            nn.Linear(256,128),
            nn.Tanh(),
            nn.Linear(128,64),
            nn.Tanh(),
            nn.Linear(64,1),
            nn.Sigmoid()
    )

    def forward(self,xb):
        out = self.network(xb)
        return out
  
    def training_step(self,batch):
        features,label = batch
#         features.to_device
#         label.to_device
        out = self(features)
        loss = F.binary_cross_entropy(out,label)
        return loss

    def validation_step(self,batch):
        features,label = batch
#         features.to_device
#         label.to_device
        out = self(features)
        loss = F.binary_cross_entropy(out,label)
        acc = accuracy(out,label)
        return {"val_loss": loss.detach(),"val_acc": acc}

    def validation_epoch_end(self,outputs):
        batch_loss = [x['val_loss'] for x in outputs]
        epoch_loss = torch.stack(batch_loss).mean()
        batch_acc = [x['val_acc'] for x in outputs]
        epoch_acc = torch.stack(batch_acc).mean()
        return {"val_loss":epoch_loss.item(),"val_acc":epoch_acc.item()}

    def epoch_end(self,num_epoch,results):
        print("num_epoch: {}, train_loss: {:.2f}, val_loss: {:.2f}, val_acc: {:.2f}".format(num_epoch+1,results['train_loss'],results['val_loss'], results['val_acc']))

In [43]:
def accuracy(out,label):
    out = (out>0.5)
    pred = (out == label).sum()
    return pred/out.shape[0]

def evaluate(model,val_loader):
    outputs = [model.validation_step(batch) for batch in val_loader]
    return model.validation_epoch_end(outputs)

def fit(num_epochs,lr,train_loader,val_loader,model,opt_func=torch.optim.Adam):
    optimizer = opt_func(model.parameters(),lr)
    history = []
    for epoch in range(num_epochs):
        train_losses = []
        for batch in train_loader:
            loss = model.training_step(batch)
            train_losses.append(loss)
            loss.backward()
            optimizer.step()
            optimizer.zero_grad()

        results = evaluate(model,val_loader)
        train_loss = torch.stack(train_losses).mean().item()
        results['train_loss'] = train_loss
        model.epoch_end(epoch,results)
        history.append(results)
    return history

In [44]:
def get_default_device():
    """Pick GPU if available, else CPU"""
    if torch.cuda.is_available():
        return torch.device('cuda')
    else:
        return torch.device('cpu')
    
def to_device(data, device):
    """Move tensor(s) to chosen device"""
    if isinstance(data, (list,tuple)):
        return [to_device(x, device) for x in data]
    return data.to(device, non_blocking=True)

class DeviceDataLoader():
    """Wrap a dataloader to move data to a device"""
    def __init__(self, dl, device):
        self.dl = dl
        self.device = device
        
    def __iter__(self):
        """Yield a batch of data after moving it to device"""
        for b in self.dl: 
            yield to_device(b, self.device)

    def __len__(self):
        """Number of batches"""
        return len(self.dl)

# Splitting the Dataset Of Arousal:-

In [45]:
x_train,x_test,y_train,y_test=train_test_split(X,np.array(arousal_labels), test_size=0.2,random_state=42)

In [46]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
x_train = scaler.fit_transform(x_train)
x_test = scaler.fit_transform(x_test)

In [47]:
train_tensor = torch.tensor(x_train,dtype=torch.float32)
train_label = torch.tensor(y_train,dtype=torch.float32)

test_tensor =torch.tensor(x_test,dtype=torch.float32)
test_label = torch.tensor(y_test,dtype=torch.float32)

In [48]:
train_label=train_label.unsqueeze(1)
test_label=test_label.unsqueeze(1)

In [49]:
train_tensor=train_tensor.reshape(train_tensor.shape[0],1,10,7)
test_tensor=test_tensor.reshape(test_tensor.shape[0],1,10,7)

In [50]:
train_tensor.shape

torch.Size([118372, 1, 10, 7])

In [51]:
test_tensor.shape

torch.Size([29594, 1, 10, 7])

In [52]:
train_arousal = TensorDataset(train_tensor, train_label)

In [53]:
test_arousal=TensorDataset(test_tensor, test_label)

In [54]:
device = get_default_device()
device

device(type='cuda')

In [55]:
batch_size = 32
train_loader =  DataLoader(train_arousal,batch_size=batch_size,shuffle=True)
test_loader =  DataLoader(test_arousal,batch_size=batch_size,shuffle=True)

In [56]:
train_loader=DeviceDataLoader(train_loader,device)
test_loader=DeviceDataLoader(test_loader,device)

In [57]:
arousal_model= SFTT()
arousal_model= arousal_model.to(device)

In [58]:
history = fit(10,0.001,train_loader,test_loader,arousal_model)

num_epoch: 1, train_loss: 0.14, val_loss: 0.09, val_acc: 0.96
num_epoch: 2, train_loss: 0.08, val_loss: 0.06, val_acc: 0.97
num_epoch: 3, train_loss: 0.06, val_loss: 0.05, val_acc: 0.97
num_epoch: 4, train_loss: 0.05, val_loss: 0.05, val_acc: 0.98
num_epoch: 5, train_loss: 0.05, val_loss: 0.05, val_acc: 0.98
num_epoch: 6, train_loss: 0.04, val_loss: 0.04, val_acc: 0.98
num_epoch: 7, train_loss: 0.03, val_loss: 0.03, val_acc: 0.99
num_epoch: 8, train_loss: 0.03, val_loss: 0.03, val_acc: 0.99
num_epoch: 9, train_loss: 0.03, val_loss: 0.03, val_acc: 0.99
num_epoch: 10, train_loss: 0.02, val_loss: 0.03, val_acc: 0.99
